# Time Series Classification Labeler - Schema Demo

This notebook demonstrates the database schema and how each table connects to the frontend UI.

In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path
from datetime import datetime

# Connect to the database
DB_PATH = Path("../backend/labeler.db")
conn = sqlite3.connect(DB_PATH)

print(f"Connected to: {DB_PATH.resolve()}")

## Schema Overview

```
┌─────────────┐     ┌─────────────┐     ┌──────────────────┐
│   devices   │────<│  rle_runs   │────<│  timeseries_data │
└─────────────┘     └─────────────┘     └──────────────────┘
                           │
                           │
                    ┌──────┴──────┐
                    ▼             │
              ┌─────────────┐     │
              │labels_votes │     │
              └─────────────┘     │
                    ▲             │
          ┌─────────┴─────────┐   │
          │                   │   │
    ┌─────────┐         ┌─────────┐
    │  users  │         │ models  │
    └─────────┘         └─────────┘
```

| Table | Purpose | Frontend Component |
|-------|---------|--------------------|
| `devices` | Device metadata & filtering | DeviceSelector dropdown |
| `rle_runs` | RLE segments to label | RunNavigator (← → navigation) |
| `timeseries_data` | Time-series points | TimeSeriesChart (ECharts) |
| `labels_votes` | Multi-labeler votes | LabelButtons + StatusIndicator |
| `users` | Labeler names | DeviceSelector "Your Name" dropdown |
| `models` | Model type definitions | DeviceSelector "Model Type" dropdown |

---
## 1. Devices Table

**Purpose:** Cache of devices with run counts for fast filtering.

**Frontend:** Populates the Device dropdown in `DeviceSelector.tsx`

In [ ]:
devices_df = pd.read_sql("SELECT * FROM devices LIMIT 10", conn)

# Convert timestamps to readable dates
devices_df['first_seen'] = pd.to_datetime(devices_df['first_seen'], unit='s')
devices_df['last_seen'] = pd.to_datetime(devices_df['last_seen'], unit='s')

print(f"Total devices: {pd.read_sql('SELECT COUNT(*) as n FROM devices', conn).iloc[0]['n']}")
devices_df

---
## 2. RLE Runs Table

**Purpose:** Each row is an RLE segment that needs labeling. A "run" is a continuous period of operation.

**Frontend:** 
- `RunNavigator.tsx` uses ← → keys to cycle through runs
- Run metadata displayed in the UI header

In [ ]:
runs_df = pd.read_sql("""
    SELECT run_id, device_id, start_ts, end_ts, run_length, created_at 
    FROM rle_runs 
    LIMIT 10
""", conn)

# Convert timestamps
runs_df['start_time'] = pd.to_datetime(runs_df['start_ts'], unit='s')
runs_df['end_time'] = pd.to_datetime(runs_df['end_ts'], unit='s')
runs_df['duration_min'] = (runs_df['run_length'] / 60).round(1)

print(f"Total runs: {pd.read_sql('SELECT COUNT(*) as n FROM rle_runs', conn).iloc[0]['n']}")
runs_df[['run_id', 'device_id', 'start_time', 'end_time', 'duration_min']]

---
## 3. Timeseries Data Table

**Purpose:** The actual time-series values for each run. One row per data point.

**Frontend:** 
- `TimeSeriesChart.tsx` renders this as an ECharts line chart
- Supports zoom, pan, and Y-axis scaling (Auto/Full)

In [ ]:
# Get a sample run
sample_run = pd.read_sql("SELECT run_id FROM rle_runs LIMIT 1", conn).iloc[0]['run_id']

ts_df = pd.read_sql(f"""
    SELECT id, run_id, ts, value 
    FROM timeseries_data 
    WHERE run_id = '{sample_run}'
    ORDER BY ts
""", conn)

ts_df['timestamp'] = pd.to_datetime(ts_df['ts'], unit='s')

print(f"Run: {sample_run}")
print(f"Data points: {len(ts_df)}")
print(f"Value range: {ts_df['value'].min():.2f} - {ts_df['value'].max():.2f}")
ts_df.head(10)

In [ ]:
# Visualize the timeseries (like the frontend chart)
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(ts_df['timestamp'], ts_df['value'], linewidth=0.8)
ax.set_xlabel('Time')
ax.set_ylabel('Value')
ax.set_title(f'Timeseries for {sample_run}')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## 4. Labels Votes Table

**Purpose:** Multi-labeler voting system. Each labeler can vote on each run independently.

**Frontend:**
- `LabelButtons.tsx` - Press 1/2/3 to submit a vote
- `StatusIndicator.tsx` - Shows current label status and vote count

**Labels:**
| Key | Label |
|-----|-------|
| 1 | Heating |
| 2 | Cooling |
| 3 | Segmentation Error |

In [ ]:
labels_df = pd.read_sql("""
    SELECT run_id, model_type, labeler, label, labeled_at, notes
    FROM labels_votes
    ORDER BY labeled_at DESC
    LIMIT 10
""", conn)

# Map label numbers to names
label_map = {1: 'Heating', 2: 'Cooling', 3: 'Segmentation Error'}
labels_df['label_name'] = labels_df['label'].map(label_map)

total_labels = pd.read_sql('SELECT COUNT(*) as n FROM labels_votes', conn).iloc[0]['n']
print(f"Total label votes: {total_labels}")

if len(labels_df) > 0:
    display(labels_df)
else:
    print("No labels yet - start labeling in the UI!")

In [ ]:
# Label distribution (if any labels exist)
if total_labels > 0:
    label_counts = pd.read_sql("""
        SELECT label, COUNT(*) as count
        FROM labels_votes
        GROUP BY label
        ORDER BY label
    """, conn)
    label_counts['label_name'] = label_counts['label'].map(label_map)
    
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.bar(label_counts['label_name'], label_counts['count'], color=['#ef4444', '#3b82f6', '#f59e0b'])
    ax.set_xlabel('Label')
    ax.set_ylabel('Count')
    ax.set_title('Label Distribution')
    plt.tight_layout()
    plt.show()

---
## 5. Users Table

**Purpose:** Registered labelers. New users can be created from the UI.

**Frontend:** `DeviceSelector.tsx` - "Your Name" searchable dropdown with create option

In [ ]:
users_df = pd.read_sql("SELECT * FROM users ORDER BY user_name", conn)

print(f"Total users: {len(users_df)}")
if len(users_df) > 0:
    display(users_df)
else:
    print("No users yet - create one in the UI!")

---
## 6. Models Table

**Purpose:** Model types being labeled. Allows labeling the same runs for different model versions.

**Frontend:** `DeviceSelector.tsx` - "Model Type" searchable dropdown with create option

In [ ]:
models_df = pd.read_sql("SELECT * FROM models ORDER BY model_name", conn)

print(f"Total models: {len(models_df)}")
models_df

---
## Relationships: Joining Tables

This query shows how the tables connect - getting runs with their device info and label counts.

In [ ]:
joined_df = pd.read_sql("""
    SELECT 
        r.run_id,
        r.device_id,
        d.run_count as device_total_runs,
        datetime(r.start_ts, 'unixepoch') as start_time,
        ROUND(r.run_length / 60.0, 1) as duration_min,
        COUNT(DISTINCT l.labeler) as labeler_count,
        GROUP_CONCAT(DISTINCT l.labeler) as labelers
    FROM rle_runs r
    JOIN devices d ON r.device_id = d.device_id
    LEFT JOIN labels_votes l ON r.run_id = l.run_id
    GROUP BY r.run_id
    ORDER BY r.start_ts DESC
    LIMIT 15
""", conn)

joined_df

---
## Frontend Component Mapping

| Component | Data Source | Key Features |
|-----------|-------------|--------------|
| `DeviceSelector.tsx` | `devices`, `users`, `models` | Date range filter, random sampling, user/model creation |
| `RunNavigator.tsx` | `rle_runs` | ← → navigation, progress indicator |
| `TimeSeriesChart.tsx` | `timeseries_data` | ECharts, zoom/pan, Auto/Full Y-axis scaling |
| `LabelButtons.tsx` | `labels_votes` | 1/2/3 hotkeys, submit votes |
| `StatusIndicator.tsx` | `labels_votes` | Vote count, consensus indicator |

In [ ]:
# Cleanup
conn.close()
print("Done!")